In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import hamming_loss
import warnings
warnings.filterwarnings("ignore")

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

### a) Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. Choose 70% of the data randomly as the training set.

In [2]:
df = pd.read_csv('../data/Frogs_MFCCs.csv')

In [3]:
train, test = train_test_split(df, test_size = 0.3)

In [4]:
train.reset_index(drop=True, inplace = True)
train_x = train.drop(columns = ['Family','Genus','Species','RecordID'])
train_y_family = train['Family']
train_y_genus = train['Genus']
train_y_species = train['Species']
train_x

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_13,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22
0,1.0,0.678890,0.627786,0.417648,0.006501,0.047394,-0.013962,0.171826,0.164646,-0.348734,...,-0.147417,-0.272056,0.250713,0.111075,-0.126669,-0.082302,0.013090,0.145827,0.013956,-0.228808
1,1.0,0.592463,0.594306,0.614234,-0.062790,-0.002779,0.406548,0.042052,-0.290214,0.121748,...,-0.097734,0.186441,0.035633,-0.163825,0.062933,0.034034,-0.090153,-0.029221,0.156980,0.051274
2,1.0,0.668390,0.730198,0.580968,-0.162741,-0.081930,0.432132,0.032331,-0.255920,0.185829,...,-0.051825,0.216703,0.085594,-0.077685,0.044806,0.058034,-0.117957,-0.023452,0.061707,-0.061029
3,1.0,0.499485,0.871131,0.130839,-0.057632,0.228049,0.113412,0.030765,-0.062252,0.114443,...,0.048553,0.036052,0.005987,-0.015289,0.014579,0.060369,-0.038024,0.027430,0.060819,-0.017502
4,1.0,0.528867,0.177182,0.480349,0.235936,0.058119,-0.113591,0.109672,0.269437,0.018172,...,0.304922,0.031393,-0.283915,-0.133012,0.144055,0.104321,-0.034758,-0.149859,0.041324,0.173693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5031,1.0,0.158996,0.051412,0.444340,0.187838,-0.044646,-0.190111,0.022224,0.237797,0.098919,...,0.262036,-0.139650,-0.212224,0.075515,0.140946,-0.070769,-0.157801,-0.097470,0.141041,0.213570
5032,1.0,0.411093,0.361243,0.599813,0.228860,0.046402,-0.122935,0.048862,0.297221,0.062370,...,0.392131,-0.130987,-0.300936,0.139467,0.261435,0.057003,-0.095145,-0.161296,0.025276,0.146362
5033,1.0,0.143910,0.215908,0.614139,0.245914,0.051628,-0.120428,0.065981,0.240651,0.011891,...,0.285553,-0.245838,-0.171299,0.247508,0.183207,-0.119715,-0.184234,-0.067429,0.161457,0.145647
5034,1.0,0.161899,0.055097,0.532344,0.153751,0.038912,-0.199959,-0.062128,0.231903,0.062480,...,0.394535,-0.090245,-0.304619,0.038661,0.233265,0.057278,-0.118224,-0.200734,0.015283,0.213167


In [5]:
test.reset_index(drop=True, inplace = True)
test_x = test.drop(columns = ['Family','Genus','Species','RecordID'])
#test_y_family = test[['Family']]
#test_y_genus = test['Genus'].tolist()
#test_y_species = test['Species'].tolist()
#type(test_y_family)

### b) Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-label classification is to train a classifier for each label (binary relevance). We first try this approach:

#### i) Research exact match and hamming score/ loss methods for evaluating multilabel classification and use them in evaluating the classifiers in this problem

Exact match

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

Hamming loss is the fraction of wrong labels to the total number of labels. In multi-class classification, hamming loss is calculated as the hamming distance between y_true and y_pred. 

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.hamming_loss.html


#### ii) Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation. You are welcome to try to solve the problem with both standardized 2 and raw attributes and report the results.

In [6]:
from sklearn import svm
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import hamming_loss

In [7]:
def svc_gaussian (column_name):
    train_y = train[column_name]
    test_y = test[[column_name]]
    clf = svm.SVC(kernel = 'rbf')
    C_list = []
    k = range(-3,6)#-3,6
    for i in k:
        C_list.append(10**(i))
    params = {'C':C_list,'gamma': np.linspace(0.1,2,20)}#(0.1,2,20)
    grid = GridSearchCV(clf, param_grid = params, cv = 10)
    grid.fit(train_x, train_y)
    pred_y = grid.predict(test_x)
    optimal_estimator = grid.best_estimator_
    optimal_params = grid.best_params_
    hamming_los = hamming_loss(test_y,pred_y)
    #print ('hamming_loss:', hamming_los)
    correct = 0
    test_y_ = test[column_name].tolist()
    for i in range(len(test_y_)):
        if test_y_[i] == pred_y[i]:
            correct+=1
    exact_match = correct/len(test_y_)
    #print ('exact_match:', exact_match)
    return ('optimal_params:',optimal_params,'hamming_loss:', hamming_los,'exact_match:', exact_match)

In [8]:
svc_gaussian('Family')

('optimal_params:',
 {'C': 10, 'gamma': 1.9},
 'hamming_loss:',
 0.008800370541917554,
 'exact_match:',
 0.9911996294580825)

In [9]:
svc_gaussian('Genus')

('optimal_params:',
 {'C': 10, 'gamma': 1.3},
 'hamming_loss:',
 0.010189902732746642,
 'exact_match:',
 0.9898100972672533)

In [10]:
svc_gaussian('Species')

('optimal_params:',
 {'C': 10, 'gamma': 1.0999999999999999},
 'hamming_loss:',
 0.010189902732746642,
 'exact_match:',
 0.9898100972672533)

#### iii) Repeat 1(b)ii with L1-penalized SVMs. Remember to standardize4 the attributes. Determine the weight of the SVM penalty using 10 fold cross validation.

In [11]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler().fit(train_x)
#mean = scaler.mean_
#train_x = scaler.transform(train_x)
#test_x = scaler.transform(test_x)

In [12]:
def svc_L1_penalized (column_name):
    train_y = train[column_name]
    test_y = test[[column_name]]
    clf = svm.LinearSVC(penalty = 'l1', dual=False)
    C_list = []
    k = range(-3,6)#-3,6
    for i in k:
        C_list.append(10**(i))
    params = {'C':C_list}
    grid = GridSearchCV(svm.LinearSVC(penalty = 'l1', dual=False), param_grid = params, cv = 10)
    grid.fit(train_x, train_y)
    pred_y = grid.predict(test_x)
    optimal_estimator = grid.best_estimator_
    optimal_params = grid.best_params_
    hamming_los = hamming_loss(test_y,pred_y)
    #print ('hamming_loss:', hamming_los)
    correct = 0
    test_y_ = test[column_name].tolist()
    for i in range(len(test_y_)):
        if test_y_[i] == pred_y[i]:
            correct+=1
    exact_match = correct/len(test_y_)
    #print ('exact_match:', exact_match)
    return ('optimal_params:',optimal_params,'hamming_loss:', hamming_los,'exact_match:', exact_match)

In [13]:
svc_L1_penalized('Family')

('optimal_params:',
 {'C': 10},
 'hamming_loss:',
 0.06484483557202408,
 'exact_match:',
 0.9351551644279759)

In [14]:
svc_L1_penalized('Genus')

('optimal_params:',
 {'C': 100000},
 'hamming_loss:',
 0.048170449282075034,
 'exact_match:',
 0.951829550717925)

In [15]:
svc_L1_penalized('Species')

('optimal_params:',
 {'C': 100},
 'hamming_loss:',
 0.03844372394627142,
 'exact_match:',
 0.9615562760537286)

#### iv) Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.

In [16]:
from imblearn.over_sampling import SMOTE


In [17]:
def svc_L1_SMOTE (column_name):
    smt = SMOTE(random_state=42)
    train_x_smote, train_y_smote = smt.fit_resample(train_x,train[column_name])
    test_y = test[[column_name]]
    
    clf = svm.LinearSVC(penalty = 'l1', dual=False)
    C_list = []
    k = range(-3,6)#-3,6
    for i in k:
        C_list.append(10**(i))
    params = {'C':C_list}
    grid = GridSearchCV(svm.LinearSVC(penalty = 'l1', dual=False), param_grid = params, cv = 10)
    grid.fit(train_x_smote, train_y_smote)
    pred_y = grid.predict(test_x)
    optimal_estimator = grid.best_estimator_
    optimal_params = grid.best_params_
    hamming_los = hamming_loss(test_y,pred_y)
    #print ('hamming_loss:', hamming_los)
    correct = 0
    test_y_ = test[column_name].tolist()
    for i in range(len(test_y_)):
        if test_y_[i] == pred_y[i]:
            correct+=1
    exact_match = correct/len(test_y_)
    #print ('exact_match:', exact_match)
    return ('optimal_params:',optimal_params,'hamming_loss:', hamming_los,'exact_match:', exact_match)

In [18]:
svc_L1_SMOTE('Family')

('optimal_params:',
 {'C': 10000},
 'hamming_loss:',
 0.0745715609078277,
 'exact_match:',
 0.9254284390921723)

In [19]:
svc_L1_SMOTE('Genus')

('optimal_params:',
 {'C': 100000},
 'hamming_loss:',
 0.0810560444650301,
 'exact_match:',
 0.9189439555349699)

In [20]:
svc_L1_SMOTE('Species')

('optimal_params:',
 {'C': 1000},
 'hamming_loss:',
 0.04353867531264474,
 'exact_match:',
 0.9564613246873552)